In [2]:
!pip -q install textstat rouge-score bert-score spacy evaluate lmqg datasets transformers accelerate bitsandbytes xlsxwriter chromadb
import nltk

nltk.download('punkt_tab')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
import pandas as pd
import json
import textwrap

# Set LOCAL variable values
# Article (data) selection
main_data = json.load(open("wikinews_data.json", 'r'))
for i, title in enumerate([d['title'] for d in main_data]):
  print(f'{i}: {title}')

# Change the index for new article
ARTICLE_INDEX = 2
data = main_data[ARTICLE_INDEX]
print(f"\nWill be working with '{data['title']}' article data!")

human_article = data['article_text']
chroma_collection_name = data['chroma_collection_name']
source_data = data['source_data']
chroma_db_path = "chroma_db"
model_path = "models"
plain_llm_article = data['plain_llm_article']
rag_llm_article = data['rag_llm_article']

# Prepare dictionary
articles = {
    "Human": human_article,
    "LLM_Plain": plain_llm_article,
    "LLM_RAG": rag_llm_article
}

# Previews
wrapper = lambda text: textwrap.fill(text, width=100)
print(f"\nHuman Article -- \n")
print(wrapper(human_article))
print(f"Word Count: {data['word_count']}")

print(f"\nPlain LLM Article -- \n")
print(wrapper(plain_llm_article))
print(f"Word Count: {len(plain_llm_article.split())}")

print(f"\nRAG LLM Article -- \n")
print(wrapper(rag_llm_article))
print(f"Word Count: {len(rag_llm_article.split())}")

0: US: Tulsa residents approve $814 million infrastructure package
1: 78th British Academy Film Awards held in London
2: Ryan Gosling cast in upcoming Star Wars film
3: Thai officials seize 238 tons of illegal e-waste at Bangkok port
4: 20-year-old astrophotographer captures rare solar eclipse on Saturn
5: Scientists discover seagrass off Australia is world's largest plant
6: India defeats New Zealand to win 2025 Champions Trophy
7: Researchers film colossal squid in its natural habitat for the first time
8: SpaceX will return stranded astronauts in February 2025, NASA announces
9: Microsoft, Nware sign 10-year cloud gaming deal
10: United Kingdom buries Queen Elizabeth II after state funeral
11: UK heavy metal band Black Sabbath announces final performance with original lineup
12: GSK rejects three Unilever bids to buy consumer healthcare arm, says unit was 'fundamentally undervalued'
13: FIFA World Cup 2018 Last 16: France, Uruguay send Argentina, Portugal home
14: European Union to 

In [5]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already present
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd

def compression_ratio_scores(article_dict, source_text):
    """
    Calculates the compression ratio (article length / source length) for all articles.
    """
    results = {}
    source_len = len(source_text.split())

    for name, text in article_dict.items():
        article_len = len(text.split())
        ratio = article_len / source_len if source_len > 0 else 0.0
        results[name] = {
            "Compression Ratio": ratio
        }

    return pd.DataFrame(results).T

# Run Compression Ratio Test
compression_df = compression_ratio_scores(articles, source_data)
display(compression_df)

,Compression Ratio
Human,0.138663
LLM_Plain,0.116907
LLM_RAG,0.110092


In [8]:
import pandas as pd

def novel_ngram_scores(article_dict, source_text):
    """
    Calculates the percentage of novel words in each article not present in the source.
    """
    results = {}
    source_words = set(w for w in source_text.lower().split() if w not in stopwords)

    for name, text in article_dict.items():
        gen_words = set(w for w in text.lower().split() if w not in stopwords)
        novel_words = gen_words - source_words
        total_gen_words = len(gen_words)

        percentage = (len(novel_words) / total_gen_words) * 100 if total_gen_words > 0 else 0.0
        results[name] = {
            "Novel N-gram %": percentage
        }

    return pd.DataFrame(results).T

# Run Novel n-gram percentage test
novel_ngram_df = novel_ngram_scores(articles, source_data)
display(novel_ngram_df)

,Novel N-gram %
Human,34.285714
LLM_Plain,17.977528
LLM_RAG,17.171717


In [ ]:
import textstat

def readability_scores(article_dict):
    results = {}
    for name, text in article_dict.items():
        results[name] = {
            "Flesch Reading Ease": textstat.flesch_reading_ease(text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(text),
            "Gunning Fog": textstat.gunning_fog(text),
            "SMOG Index": textstat.smog_index(text)
        }
    return pd.DataFrame(results).T


# Run FK readability test
readability_df = readability_scores(articles)
display(readability_df)

,Flesch Reading Ease,Flesch-Kincaid Grade,Gunning Fog,SMOG Index
Human,42.613203,13.886559,16.931607,15.247665
LLM_Plain,47.383650,12.481777,14.950012,13.936534
LLM_RAG,48.673442,11.213074,13.636364,13.246249


In [9]:
def distinct_n_scores(article_dict):
    rows = []
    for name, text in article_dict.items():
        tokens = [w for w in text.lower().split() if w not in stopwords]
        def distinct_n(n):
            if len(tokens) < n: return 0.0
            ngrams = [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
            return len(set(ngrams)) / max(1, len(ngrams))
        rows.append({
            "version": name,
            "Distinct-1": distinct_n(1),
            "Distinct-2": distinct_n(2)
        })
    return pd.DataFrame(rows).set_index("version")


# Run Disctinct score test
distinct_df = distinct_n_scores(articles)
display(distinct_df)

,Distinct-1,Distinct-2
version,,
Human,0.693069,0.920000
LLM_Plain,0.831776,0.962264
LLM_RAG,0.792000,0.983871


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re

def get_tfidf_keywords(source_text, keyword_ratio=0.10):
    """
    Extracts the top N most important keywords from the source text using TF-IDF.
    This defines the objective 'gold standard' of key terms.
    """
    # Clean the text: keep only words with 3+ letters [a-zA-Z]
    words_only = re.findall(r'\b\w{3,}\b', source_text.lower())
    cleaned_text = ' '.join(words_only)
    unique_words = set(words_only)
    n_unique = len(unique_words)

    # Calculate dynamic n_keywords - ensure at least 1 keyword
    n_keywords = max(1, int(n_unique * keyword_ratio))

    # Use TF-IDF.
    vectorizer = TfidfVectorizer(stop_words='english', max_features=n_keywords)
    tfidf_matrix = vectorizer.fit_transform([cleaned_text])

    feature_names = vectorizer.get_feature_names_out()
    keywords_set = set(feature_names)

    print(f"Source has {n_unique} unique words. Extracting top {n_keywords} keywords ({keyword_ratio*100}%).")
    return keywords_set

def calculate_keyword_metrics(article_words, source_keywords):
    """
    Helper function to calculate Precision, Recall, and F1 for one article.
    """
    article_keywords = source_keywords.intersection(article_words)
    other_words = article_words - source_keywords

    # True Positives: Keywords from the set that are in the article
    TP = len(article_keywords)
    # False Positives: Words in the article that are NOT in the keyword set
    FP = len(other_words)
    # False Negatives: Keywords from the set that are NOT in the article
    FN = len(source_keywords - article_words)

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    return TP, FP, FN, precision, recall, f1

def keyword_overlap_tfidf(article_dict, source_text, keyword_ratio):
    """
    Main function to calculate Keyword Overlap for all articles against
    TF-IDF keywords derived from the source.
    """
    # Get the keyword set from the Source Data
    source_keywords = get_tfidf_keywords(source_text, keyword_ratio)
    print(f"Top {len(source_keywords)} Keywords from Source (via TF-IDF):")
    print(source_keywords)

    # Pre-tokenize all articles and source for efficiency
    tokenized_articles = {}
    for name, text in article_dict.items():
        words = set(re.findall(r'\b\w{3,}\b', text.lower()))
        tokenized_articles[name] = words

    results_data = {}
    for name, words in tokenized_articles.items():
        TP, FP, FN, precision, recall, f1 = calculate_keyword_metrics(words, source_keywords)

        results_data[name] = {
            "TP": TP,
            "FP": FP,
            "FN": FN,
            "Keyword Precision": precision,
            "Keyword Recall": recall,
            "Keyword F1": f1,
            "Keywords Found": ', '.join(sorted(source_keywords.intersection(words))),
            "Keywords Missing": ', '.join(sorted(source_keywords - words))
        }

    df = pd.DataFrame(results_data).T

    return df


# Run Keyword Overlap Test
print(data['word_count_source_data'])
keyword_overlap_df = keyword_overlap_tfidf(articles, source_data, keyword_ratio=0.10)
display(keyword_overlap_df)

3911
Source has 1037 unique words. Extracting top 103 keywords (10.0%).
Top 103 Keywords from Source (via TF-IDF):
{'new', 'development', 'measures', '2050', 'ccc', 'biden', 'gas', 'law', 'framework', 'communication', 'today', 'states', 'parliament', 'poland', 'emergency', 'possible', 'end', 'need', 'key', 'net', 'commission', 'indicators', 'effort', 'government', 'plans', 'rising', 'binding', 'levels', 'supply', 'costs', 'drive', 'competitive', '1990', 'member', 'time', 'emissions', 'ambition', 'level', 'world', 'agreement', 'reduction', 'targets', 'efficiency', 'environmental', 'minister', 'progress', 'based', 'effective', 'target', 'change', 'path', 'paris', 'global', 'cost', 'set', 'energy', 'low', 'years', 'focus', 'market', 'climate', 'ensure', 'secure', 'europe', 'investment', 'prices', 'national', 'european', 'stability', 'leaders', 'ambitious', 'international', 'council', 'policy', 'said', '2030', 'trading', 'agreed', 'governance', 'achieve', 'commitments', 'carbon', 'rise', '

,TP,FP,FN,Keyword Precision,Keyword Recall,Keyword F1,Keywords Found,Keywords Missing
Human,51,197,52,0.205645,0.495146,0.290598,"1990, 2030, 2050, action, agreed, agreement, a...","achieve, based, biden, binding, ccc, commitmen..."
LLM_Plain,54,146,49,0.27,0.524272,0.356436,"1990, 2030, 2050, achieve, action, agreed, agr...","based, commitments, communication, competitive..."
LLM_RAG,49,145,54,0.252577,0.475728,0.329966,"1990, 2030, achieve, action, agreed, agreement...","2050, ambitious, based, biden, binding, coal, ..."


In [10]:
import pandas as pd

def jaccard_similarity_scores(article_dict, reference_key="Human"):
    """
    Calculates Jaccard Similarity between each article and the reference article.
    """
    results = {}
    ref_text = article_dict[reference_key]
    ref_words = set([w for w in ref_text.lower().split() if w not in stopwords])

    for name, text in article_dict.items():
        if name == reference_key:
            # The reference compared to itself will have perfect similarity
            similarity = 1.0
        else:
            comp_words = set([w for w in text.lower().split() if w not in stopwords])
            intersection = ref_words.intersection(comp_words)
            union = ref_words.union(comp_words)

            similarity = len(intersection) / len(union) if len(union) > 0 else 0.0

        results[name] = {
            f"Jaccard Similarity (vs {reference_key})": similarity
        }

    return pd.DataFrame(results).T

# Run Jaccard Similarity Score
jaccard_df = jaccard_similarity_scores(articles, "Human")
display(jaccard_df)

,Jaccard Similarity (vs Human)
Human,1.000000
LLM_Plain,0.232558
LLM_RAG,0.190141


In [ ]:
from rouge_score import rouge_scorer

def rouge_scores(article_dict, reference_key="Human"):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = {}
    ref_text = article_dict[reference_key]
    for name, text in article_dict.items():
        if name == reference_key:
            continue
        s = scorer.score(ref_text, text)
        results[name] = {
            f"ROUGE-1 F1 (vs {reference_key})": s["rouge1"].fmeasure,
            f"ROUGE-2 F1 (vs {reference_key})": s["rouge2"].fmeasure,
            f"ROUGE-L F1 (vs {reference_key})": s["rougeL"].fmeasure
        }
    return pd.DataFrame(results).T


# Run ROUGE Score test
rouge_df = rouge_scores(articles)
display(rouge_df)

,ROUGE-1 F1 (vs Human),ROUGE-2 F1 (vs Human),ROUGE-L F1 (vs Human)
LLM_Plain,0.597194,0.257028,0.250501
LLM_RAG,0.573183,0.221538,0.237462


In [ ]:
import torch
import gc

# This ensures GPU has enough Memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoConfig
from bert_score import score as bert_score

bert_model_name = "roberta-large"
bert_model_dir = f"{model_path}/bert_score_model"

# Download and save if not already present
if not os.path.exists(bert_model_dir):
  os.makedirs(bert_model_dir, exist_ok=True)
  print(f"Downloading {bert_model_name} model to {bert_model_dir} ......")
  tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
  model = AutoModel.from_pretrained(bert_model_name)
  tokenizer.save_pretrained(bert_model_dir)
  model.save_pretrained(bert_model_dir)
else:
  print(f"Using Local BERTScore model at: {bert_model_dir}")

# Read number of layers from local config
cfg = AutoConfig.from_pretrained(bert_model_dir)
NUM_LAYERS = getattr(cfg, "num_hidden_layers", None)
if NUM_LAYERS is None:
  NUM_LAYERS = 24

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loaded model from {bert_model_dir} successfully on {device.upper()}!")

Using Local BERTScore model at: /content/drive/MyDrive/Colab Notebooks/Dissertation/models/bert_score_model
Loaded model from /content/drive/MyDrive/Colab Notebooks/Dissertation/models/bert_score_model successfully on CUDA!


In [ ]:
def bert_scores(article_dict, reference_key="Human"):
    results = {}
    ref_text = [article_dict[reference_key]]
    for name, text in article_dict.items():
        if name == reference_key:
            continue
        P, R, F1 = bert_score(
            [text],
            ref_text,
            model_type=bert_model_dir,
            num_layers=NUM_LAYERS,
            lang=None,
            device=device,
            batch_size=8,
            verbose=False,
            rescale_with_baseline=False
        )
        results[name] = {
            f"BERTScore P (vs {reference_key})": float(P[0]),
            f"BERTScore R (vs {reference_key})": float(R[0]),
            f"BERTScore F1 (vs {reference_key})": float(F1[0])
        }
    return pd.DataFrame(results).T


# Run BERTScore test
bert_df = bert_scores(articles)
display(bert_df)

,BERTScore P (vs Human),BERTScore R (vs Human),BERTScore F1 (vs Human)
LLM_Plain,0.95839,0.958395,0.958392
LLM_RAG,0.96016,0.958356,0.959257


In [ ]:
import torch
import gc

# This ensures GPU has enough Memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, pipeline

Q2_MODEL_NAME = "google/seahorse-large-q2"
Q2_LOCAL_DIR  = f"{model_path}/seahorse_q2_model"

if not os.path.exists(Q2_LOCAL_DIR):
    os.makedirs(Q2_LOCAL_DIR, exist_ok=True)
    print(f"Downloading {Q2_MODEL_NAME} to {Q2_LOCAL_DIR} ...")
    tok = AutoTokenizer.from_pretrained(Q2_MODEL_NAME)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(Q2_MODEL_NAME)
    tok.save_pretrained(Q2_LOCAL_DIR)
    mdl.save_pretrained(Q2_LOCAL_DIR)
    print("Saved locally.")
else:
    print(f"Using local Q2 model at: {Q2_LOCAL_DIR}")

device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(Q2_LOCAL_DIR, use_fast=True)

if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token or "[PAD]"

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
mdl = AutoModelForSeq2SeqLM.from_pretrained(
    Q2_LOCAL_DIR,
    device_map="auto",
    torch_dtype=dtype
)
q2_evaluator = pipeline(
    "text2text-generation",
    model=mdl,
    tokenizer=tok
)
print(f"Loaded model from {Q2_LOCAL_DIR} successfully on {device.upper()}!")

Using local Q2 model at: /content/drive/MyDrive/Colab Notebooks/Dissertation/models/seahorse_q2_model


Device set to use cuda:0


Loaded model from /content/drive/MyDrive/Colab Notebooks/Dissertation/models/seahorse_q2_model successfully on CUDA!


In [ ]:
def q2_scores(premise, hypothesis, version):
  """
  Runs Q2 (SEAHORSE) locally for each candidate vs the reference.
  Returns a DataFrame with raw labels and a simple numeric mapping.
  """
  prompt = f"premise: {premise}\nhypothesis: {hypothesis}"
  out = q2_evaluator(
      prompt,
      max_new_tokens=8,
      do_sample=False,
      pad_token_id=tok.pad_token_id
  )[0]["generated_text"]

  label = out.strip().lower()

  # simple numeric mapping
  score = 1.0 if label in ["true", "1", "1.0", True, 1, 1.0] else 0.0 if label in ['false', "0", '0.0', False, 0, 0.0] else None

  return pd.DataFrame([{
      "version": version,
      "Q2 Model Output": label,
      "Q2_score": score
  }])


# Run Q2 Test for all 3 articles
q2_human_df = q2_scores(source_data[:10000], articles['Human'], "Human")
q2_plain_df = q2_scores(source_data[:10000], articles['LLM_Plain'], "LLM_Plain")
q2_rag_df = q2_scores(source_data[:10000], articles['LLM_RAG'], "LLM_RAG")

q2_df = pd.concat([q2_human_df, q2_plain_df, q2_rag_df], ignore_index=True)
display(q2_df)

,version,Q2 Model Output,Q2_score
0,Human,1,1.0
1,LLM_Plain,1,1.0
2,LLM_RAG,1,1.0


In [ ]:
import torch
import gc

# This ensures GPU has enough Memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from huggingface_hub import login
import os, json, random
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Hugging Face login
token = open("hf_token.txt", "r").read().strip()
login(token=token, add_to_git_credential=True)

JUDGE_MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
JUDGE_LOCAL_DIR  = f"{model_path}/judge_llm_3.2"

if not os.path.exists(JUDGE_LOCAL_DIR):
    os.makedirs(JUDGE_LOCAL_DIR, exist_ok=True)
    print(f"Downloading {JUDGE_MODEL_NAME} to {JUDGE_LOCAL_DIR} ...")
    tok = AutoTokenizer.from_pretrained(JUDGE_MODEL_NAME, token=token, use_fast=False, legacy=True)
    mdl = AutoModelForCausalLM.from_pretrained(JUDGE_MODEL_NAME, token=token)
    tok.save_pretrained(JUDGE_LOCAL_DIR)
    mdl.save_pretrained(JUDGE_LOCAL_DIR)
else:
    print(f"Using local Judge model at: {JUDGE_LOCAL_DIR}")

device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(JUDGE_LOCAL_DIR, use_fast=False, legacy=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token or "[PAD]"

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
mdl = AutoModelForCausalLM.from_pretrained(
    JUDGE_LOCAL_DIR,
    device_map="auto",
    torch_dtype=dtype
)

judge_llm = pipeline("text-generation", model=mdl, tokenizer=tok)

print(f"Loaded model from {JUDGE_LOCAL_DIR} successfully on {device.upper()}!")

Using local Judge model at: /content/drive/MyDrive/Colab Notebooks/Dissertation/models/judge_llm_3.2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Loaded model from /content/drive/MyDrive/Colab Notebooks/Dissertation/models/judge_llm_3.2 successfully on CUDA!


In [ ]:
import re
import random
from nltk.tokenize import sent_tokenize

# Form Snippets -- random n sentences from the source data
sents = [s.strip() for s in sent_tokenize(source_data) if s.strip()]
random.seed(42)  # reproducible
snippets = random.sample(sents, 20)
print(f"snippets --")
for s in snippets:
  print(s)

snippets = "\n".join(snippets)


def remap(df, article_a_label="1", article_b_label="2"):
    """
    Remap '1' and '2' labels in a pairwise comparison DataFrame
    to human-readable labels.
    """
    mapping = {"1": article_a_label, "2": article_b_label, "tie": "tie"}

    df = df.copy()
    df["winner"] = df["winner"].map(mapping).fillna(df["winner"])
    if 'factual_accuracy_winner' in df.columns:
      df["factual_accuracy_winner"] = df["factual_accuracy_winner"].map(mapping).fillna(df["factual_accuracy_winner"])
    if 'coverage_winner' in df.columns:
      df["coverage_winner"] = df["coverage_winner"].map(mapping).fillna(df["coverage_winner"])
    if 'coherence_winner' in df.columns:
      df["coherence_winner"] = df["coherence_winner"].map(mapping).fillna(df["coherence_winner"])
    if 'neutrality_winner' in df.columns:
      df["neutrality_winner"] = df["neutrality_winner"].map(mapping).fillna(df["neutrality_winner"])
    if 'readability_winner' in df.columns:
      df["readability_winner"] = df["readability_winner"].map(mapping).fillna(df["readability_winner"])

    return df


def pairwise_comparison_scores(title, snippets, article_text_a, article_text_b):
  # # Randomly swap articles to prevent position bias
  # swapped = random.random() > 0.5
  # article_a = article_text_b if swapped else article_text_a
  # article_b = article_text_a if swapped else article_text_b

  # Format prompt
  prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert judge comparing two news articles. Your only goal is to determine which article is better based solely on the CONTEXT provided.

<|eot_id|><|start_header_id|>user<|end_header_id|>

# INSTRUCTIONS:
1.  **FIRST, CHECK FOR IDENTICAL ARTICLES:** If Article 1 and Article 2 have the exact same text, output a tie immediately.
2.  **EVALUATE INDEPENDENTLY:** Read each article carefully against the CONTEXT. Do not let their order (1 or 2) influence you.
3.  **COMPARE:** Decide which article is better overall. It is okay to have a winner; a tie is only for identical quality.
4.  **OUTPUT:** Respond with valid JSON only, using the exact schema below.

# EVALUATION CRITERIA (use only these):
-   **factual_accuracy:** Does it match the CONTEXT facts without errors?
-   **coverage:** Does it include the most important information?
-   **coherence:** Is it well-structured and easy to follow?
-   **neutrality:** Is the tone objective and unbiased?
-   **readability:** Is the language clear and concise?

# CONTEXT (Your only source of truth):
{snippets}

# ARTICLE 1:
{article_text_a}

# ARTICLE 2:
{article_text_b}

# OUTPUT SCHEMA (You MUST use this exact JSON format):
{{
  "winner": "1" | "2" | "tie",
  "confidence": 1 | 2 | 3 | 4 | 5,
  "reasoning": "Your 3-sentence explanation here. Use only single quotes if needed.",
  "criteria_comparison": {{
    "factual_accuracy": "1" | "2" | "tie",
    "coverage": "1" | "2" | "tie",
    "coherence": "1" | "2" | "tie",
    "neutrality": "1" | "2" | "tie",
    "readability": "1" | "2" | "tie"
  }}
}}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

  # Get LLM response
  response = judge_llm(
      prompt,
      max_new_tokens=512,
      do_sample=False)[0]['generated_text']

  # Extract only the new generated part (remove the input prompt)
  response = response[len(prompt):].strip()

  # If response starts with assistant header, remove it
  if response.startswith('<|start_header_id|>assistant<|end_header_id|>'):
      response = response.replace('<|start_header_id|>assistant<|end_header_id|>', '').strip()

  # Extract JSON from the response
  json_start = response.find('{')
  json_end = response.rfind('}') + 1
  if json_start >= 0 and json_end > json_start:
      response = response[json_start:json_end]

  try:
      # Replace all double quotes in the reasoning part of the LLM response
      response = re.sub(
          r'("reasoning"\s*:\s*")(.*?)(?="[,}])',
          lambda m: m.group(1) + m.group(2).replace('"', "'"),
          response,
          flags=re.DOTALL
      )
      pairwise_result = json.loads(response, strict=False)

  except Exception as e:
      print("ERROR!!")
      print(response)
      pairwise_result = {"error": str(e), "raw_response": response, "winner": str(e)}
      return pd.DataFrame([pairwise_result])

  # if not swapped:
  #   # No swap occurred, labels are correct
  #   print("No Swap has occurred!")
  #   pass

  # else:
  #   # Swap occurred, so the winner labels need to be reversed
  #   print("Swap has occurred!")

  #   if pairwise_result.get("winner") == "1":
  #       pairwise_result["winner"] = "2"
  #   elif pairwise_result.get("winner") == "2":
  #       pairwise_result["winner"] = "1"

  #   # Also reverse the labels for the criteria_comparison
  #   criteria_comp = pairwise_result.get("criteria_comparison", {})
  #   for key, value in criteria_comp.items():
  #       if value == "1":
  #           criteria_comp[key] = "2"
  #       elif value == "2":
  #           criteria_comp[key] = "1"

  # Extract key metrics for the row
  row_data = {
      "winner": pairwise_result.get("winner", "error"),
      "confidence": pairwise_result.get("confidence", 0),
      "reasoning": pairwise_result.get("reasoning", ""),
  }

  # Add criteria-level winners
  if "criteria_comparison" in pairwise_result:
      criteria_comp = pairwise_result["criteria_comparison"]
      row_data.update({
          "factual_accuracy_winner": criteria_comp.get("factual_accuracy", "error"),
          "coverage_winner": criteria_comp.get("coverage", "error"),
          "coherence_winner": criteria_comp.get("coherence", "error"),
          "neutrality_winner": criteria_comp.get("neutrality", "error"),
          "readability_winner": criteria_comp.get("readability", "error")
      })

  return pd.DataFrame([row_data])

# Human vs Plain LLM
human_plain = pairwise_comparison_scores(data['title'], snippets, human_article, plain_llm_article)
human_plain = remap(human_plain, 'Human', 'Plain LLM')
human_plain = human_plain.rename(index={0: "Human_vs_Plain"}).T
print("Human vs Plain LLM comparison done!")
# display(human_plain)

plain_human = pairwise_comparison_scores(data['title'], snippets, plain_llm_article, human_article)
plain_human = remap(plain_human, 'Plain LLM', 'Human')
plain_human = plain_human.rename(index={0: "Plain_vs_Human"}).T
print("Plain LLM vs Human comparison done!")
# display(plain_human)

# Human vs RAG LLM
human_rag = pairwise_comparison_scores(data['title'], snippets, human_article, rag_llm_article)
human_rag = remap(human_rag, 'Human', 'RAG LLM')
human_rag = human_rag.rename(index={0: "Human_vs_RAG"}).T
print("Human vs RAG LLM comparison done!")
# display(human_rag)

rag_human = pairwise_comparison_scores(data['title'], snippets, rag_llm_article, human_article)
rag_human = remap(rag_human, 'RAG LLM', 'Human')
rag_human = rag_human.rename(index={0: "RAG_vs_Human"}).T
print("RAG LLM vs Human comparison done!")
# display(rag_human)

# RAG LLM vs Plain LLM
rag_plain = pairwise_comparison_scores(data['title'], snippets, rag_llm_article, plain_llm_article)
rag_plain = remap(rag_plain, 'RAG LLM', 'Plain LLM')
rag_plain = rag_plain.rename(index={0: "RAG_vs_Plain"}).T
print("RAG LLM vs Plain LLM comparison done!")
# display(rag_plain)

plain_rag = pairwise_comparison_scores(data['title'], snippets, plain_llm_article, rag_llm_article)
plain_rag = remap(plain_rag, 'Plain LLM', 'RAG LLM')
plain_rag = plain_rag.rename(index={0: "Plain_vs_RAG"}).T
print("Plain LLM vs RAG LLM comparison done!")
# display(plain_rag)

# Sanity check to test Judge when we send the exact same content
# plain_plain = pairwise_comparison_scores(data['title'], snippets, plain_llm_article, plain_llm_article)
# plain_plain = remap(plain_plain, 'Plain LLM', 'Plain LLM')
# plain_plain = plain_plain.rename(index={0: "Plain_vs_Plain"}).T
# display(plain_plain)

llm_as_a_judge_df = pd.concat([human_plain, plain_human, human_rag, rag_human, rag_plain, plain_rag], axis=1)
display(llm_as_a_judge_df)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


snippets --
MEPs, who had sought a 60% cut by 2030, also failed to significantly raise the ambitions of the EU's capitals despite the claims on Wednesday that the bloc was showing global leadership on the climate emergency.
US President Joe Biden, who will lead the meeting, rejoined the Paris climate agreement in his first day in office and has previously committed to reaching net zero emissions by 2050.
The government has adopted the new 2035 deadline for a 78% emissions cut because scientists say this is needed to keep the rise in global temperatures close to 1.5C above pre-industrial levels.
But it adds that in time fuel savings from more efficient equipment will cancel out investment costs.
Tom Burke, who chairs the environmental think tank E3G, explained what policy changes were needed to achieve the goal: 'The most important thing, I think, is for [the prime minister] to focus his policy around energy efficiency, around wind and solar, and around storage of electricity and the ma

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Human vs Plain LLM comparison done!


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


RAG LLM vs Human comparison done!


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


RAG LLM vs Plain LLM comparison done!
Plain LLM vs RAG LLM comparison done!


,Human_vs_Plain,Plain_vs_Human,Human_vs_RAG,RAG_vs_Human,RAG_vs_Plain,Plain_vs_RAG
winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM
confidence,4,4,4,4,4,4
reasoning,Article 2 presents a more comprehensive and ac...,Article 2 provides more detailed information a...,Article 2 presents a more detailed and nuanced...,Article 2 provides more detailed information a...,Article 2 presents a more comprehensive and ac...,Article 1 presents a more comprehensive and ac...
factual_accuracy_winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM
coverage_winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM
coherence_winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM
neutrality_winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM
readability_winner,Plain LLM,Human,RAG LLM,Human,Plain LLM,Plain LLM


In [ ]:
# Save all the evaluation data to one sheet
def multiple_dfs(df_list, sheet_name, file_name, spaces=2):
    with pd.ExcelWriter(file_name, engine="xlsxwriter") as writer:
        row = 0
        for dataframe in df_list:
            dataframe.to_excel(writer, sheet_name=sheet_name, startrow=row, startcol=0)
            row += len(dataframe.index) + spaces + 1

# List of dataframes
file_name = f"{data['title'].replace(' ', '_')}_eval_metrics_data.xlsx"
dfs = [
    compression_df,
    novel_ngram_df,
    readability_df,
    distinct_df,
    keyword_overlap_df,
    jaccard_df,
    rouge_df,
    bert_df,
    q2_df,
    llm_as_a_judge_df
]

multiple_dfs(dfs, "evaluation_metrics", f"/content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{file_name}", spaces=2)
print(f"Evaluation metrics saved to saved to /content/drive/MyDrive/Colab Notebooks/Dissertation/Article_{ARTICLE_INDEX}/{file_name}!")

metrics = ['Compression_Ratio_results',
           'Novel_N_Gram_results',
           'Readability_results',
           'Distinct_Score_results',
           'Keyword_Recall_results',
           'Jaccard_Similarity_results',
           'ROUGE_results',
           'BERT_results',
           'Q2_results',
           'LLM_as_a_Judge_results'
           ]

for df, metric_name in zip(dfs, metrics):
    if metric_name == "LLM_as_a_Judge_results":
        llm_results = {metric_name: df.to_dict(orient="records")}
        for index, llm_inside_result in enumerate(llm_results[metric_name]):
            if index == 0:
                llm_results[metric_name][index] = {f"{k}_winner": v for k, v in llm_inside_result.items()}
            elif index == 1:
                llm_results[metric_name][index] = {f"{k}_confidence_score": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 2:
                llm_results[metric_name][index] = {f"{k}_reasoning_text": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 3:
                llm_results[metric_name][index] = {f"{k}_factual_accuracy_winner": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 4:
                llm_results[metric_name][index] = {f"{k}_coverage_winner": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 5:
                llm_results[metric_name][index] = {f"{k}_coherence_winner": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 6:
                llm_results[metric_name][index] = {f"{k}_neutrality_winner": v for k, v in
                                                   llm_inside_result.items()}
            elif index == 7:
                llm_results[metric_name][index] = {f"{k}_readability_winner": v for k, v in
                                                   llm_inside_result.items()}

        _d = {**_d, **llm_results}
    else:
        _d = {**_d, **{metric_name: df.to_dict(orient="records")}}

    data = {**data, **_d}

main_data[ARTICLE_INDEX] = data
json.dump(main_data, open("wikinews_data.json", "w"), indent=3, force_ascii=False, orient="records")

Evaluation metrics saved to saved to /content/drive/MyDrive/Colab Notebooks/Dissertation/Article_14/European_Union_to_reduce_carbon_emissions_by_55%_of_1990_levels_by_2030_eval_metrics_data.xlsx!
